In [1]:
# import os
# import hashlib
# from PIL import Image
# import imagehash # For aHash, pHash, dHash
# import pandas as pd
# from concurrent.futures import ProcessPoolExecutor
# from collections import defaultdict
# import time
# import shutil # For robust saving

# # from tqdm import tqdm # Optional: for a nicer progress bar

# Image.MAX_IMAGE_PIXELS = None

# # --- Configuration ---
# DEFAULT_IMAGE_FOLDER = r'/blue/hulcr/gmarais/PhD/phase_1_data/20250515/output' # User's default
# OUTPUT_CSV_FILE = 'image_hashes_and_duplicates.csv'
# MAX_WORKERS = os.cpu_count()  # Use all available CPU cores
# SAVE_INTERVAL = 1000 # Number of NEWLY PROCESSED images before an interim save

# # --- Hashing Algorithms ---
# # Dhash and sha25 seem to work the best
# HASH_ALGORITHMS = {
#     'sha256': lambda img: hashlib.sha256(img.tobytes()).hexdigest(),
#     # 'ahash': lambda img: str(imagehash.average_hash(img)),
#     # 'phash': lambda img: str(imagehash.phash(img)),
#     'dhash': lambda img: str(imagehash.dhash(img)),
#     # 'whash': lambda img: str(imagehash.whash(img)),
#     # 'colorhash': lambda img: str(imagehash.colorhash(img)),
# }

# def robust_df_save(df, filepath):
#     """Saves DataFrame to a CSV file robustly using a temporary file."""
#     temp_filepath = filepath + ".tmp"
#     try:
#         df.to_csv(temp_filepath, index=False)
#         # os.replace is generally atomic, shutil.move is a good fallback/alternative
#         shutil.move(temp_filepath, filepath)
#         print(f"DataFrame successfully saved to {filepath}")
#     except Exception as e:
#         print(f"Error saving DataFrame to {filepath}: {e}")
#         if os.path.exists(temp_filepath):
#             try:
#                 os.remove(temp_filepath)
#             except Exception as e_remove:
#                 print(f"Error removing temporary file {temp_filepath}: {e_remove}")
#         return False # Indicate failure
#     return True # Indicate success

# def calculate_hashes_for_image(image_path):
#     """
#     Opens an image and calculates all specified hashes.
#     Returns a dictionary with image name and its hashes, or an error dict.
#     """
#     image_name = os.path.basename(image_path)
#     try:
#         img = Image.open(image_path).convert('RGB')
#         hashes = {'image_name': image_name}
#         for algo_name, hash_func in HASH_ALGORITHMS.items():
#             try:
#                 hashes[algo_name] = hash_func(img)
#             except Exception: # e
#                 # print(f"\nWarning: Could not compute {algo_name} for {image_path}: {e}") # Commented for cleaner progress
#                 hashes[algo_name] = None
#         return hashes
#     except FileNotFoundError:
#         # print(f"\nError: Image file not found at {image_path}") # Commented
#         return {'image_name': image_name, 'error': 'File not found'}
#     except Exception as e:
#         # print(f"\nError processing image {image_path}: {e}") # Commented
#         return {'image_name': image_name, 'error': str(e)}

# def find_png_images(folder_path):
#     """Scans the folder for PNG images."""
#     png_files = []
#     for root, _, files in os.walk(folder_path):
#         for file in files:
#             if file.lower().endswith('.png'):
#                 png_files.append(os.path.join(root, file))
#     return png_files

# def main():
#     script_start_time = time.time()

#     image_folder_input = input(f"Enter path to image folder (default: {DEFAULT_IMAGE_FOLDER}): ")
#     image_folder = image_folder_input if image_folder_input else DEFAULT_IMAGE_FOLDER

#     if not os.path.isdir(image_folder):
#         print(f"Error: Folder not found: '{image_folder}'.")
#         return

#     # --- Load Existing Data ---
#     master_df = pd.DataFrame(columns=['Image Name', 'Hash Algorithm', 'Hash Value']) # Ensure columns exist even if empty
#     processed_image_names = set()
#     if os.path.exists(OUTPUT_CSV_FILE):
#         print(f"Found existing CSV: {OUTPUT_CSV_FILE}. Loading...")
#         try:
#             master_df = pd.read_csv(OUTPUT_CSV_FILE)
#             # Ensure essential columns exist; 'Duplicates' might or might not
#             if 'Image Name' in master_df.columns:
#                  processed_image_names = set(master_df['Image Name'].unique())
#             print(f"Loaded {len(processed_image_names)} unique image names from existing CSV.")
#             # Keep only essential columns for appending new hash data for now
#             if 'Duplicates' in master_df.columns:
#                 master_df = master_df[['Image Name', 'Hash Algorithm', 'Hash Value']]
#         except pd.errors.EmptyDataError:
#             print(f"Warning: {OUTPUT_CSV_FILE} is empty. Starting fresh.")
#             master_df = pd.DataFrame(columns=['Image Name', 'Hash Algorithm', 'Hash Value'])
#         except Exception as e:
#             print(f"Error loading {OUTPUT_CSV_FILE}: {e}. Starting with an empty dataset.")
#             master_df = pd.DataFrame(columns=['Image Name', 'Hash Algorithm', 'Hash Value'])
#     else:
#         print(f"No existing CSV found at {OUTPUT_CSV_FILE}. A new one will be created.")

#     print(f"Scanning for PNG images in '{image_folder}'...")
#     all_found_image_paths = find_png_images(image_folder)
    
#     image_paths_to_process = [
#         p for p in all_found_image_paths if os.path.basename(p) not in processed_image_names
#     ]
#     total_images_to_process = len(image_paths_to_process)

#     if not image_paths_to_process:
#         print("No new PNG images found to process.")
#         if processed_image_names:
#             print("Proceeding with duplicate analysis on existing data.")
#         else:
#             print("No images to process and no existing data. Exiting.")
#             return
#     else:
#         print(f"Found {len(all_found_image_paths)} total PNG images. Need to process {total_images_to_process} new images.")
#         print(f"Starting hashing process using up to {MAX_WORKERS} workers...")

#         newly_computed_df_records = []
#         processed_in_this_run_count = 0
#         failed_during_this_run_count = 0
        
#         hashing_start_time = time.time()

#         with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
#             for result_dict in executor.map(calculate_hashes_for_image, image_paths_to_process):
#                 processed_in_this_run_count += 1
#                 if 'error' in result_dict:
#                     print(f"\nSkipped hashing for {result_dict['image_name']}: {result_dict['error']}")
#                     failed_during_this_run_count +=1
#                 elif result_dict:
#                     img_name = result_dict['image_name']
#                     for algo_name in HASH_ALGORITHMS.keys():
#                         hash_value = result_dict.get(algo_name)
#                         if hash_value is not None:
#                             newly_computed_df_records.append({
#                                 'Image Name': img_name,
#                                 'Hash Algorithm': algo_name,
#                                 'Hash Value': hash_value
#                             })
                
#                 # Progress printing
#                 progress_percent = (processed_in_this_run_count / total_images_to_process) * 100 if total_images_to_process > 0 else 0
#                 print(f"Progress: {processed_in_this_run_count}/{total_images_to_process} new images ({progress_percent:.2f}%)", end='\r')

#                 # Incremental save
#                 if newly_computed_df_records and \
#                    (processed_in_this_run_count % SAVE_INTERVAL == 0 or processed_in_this_run_count == total_images_to_process):
#                     print(f"\nSaving intermediate results... ({processed_in_this_run_count}/{total_images_to_process} processed)")
#                     temp_df = pd.DataFrame(newly_computed_df_records)
#                     master_df = pd.concat([master_df, temp_df], ignore_index=True)
#                     # Remove duplicates that might arise if script restarts and re-processes an image from a saved batch
#                     master_df.drop_duplicates(subset=['Image Name', 'Hash Algorithm', 'Hash Value'], keep='first', inplace=True) # Or keep='last'
                    
#                     robust_df_save(master_df, OUTPUT_CSV_FILE)
#                     newly_computed_df_records = [] # Reset for next batch
        
#         if total_images_to_process > 0:
#             print() # Newline after progress bar

#         hashing_duration = time.time() - hashing_start_time
#         print(f"Hashing of {processed_in_this_run_count - failed_during_this_run_count} new images (producing records) completed in {hashing_duration:.2f}s.")
#         if failed_during_this_run_count > 0:
#             print(f"{failed_during_this_run_count} images failed to process during this run.")

#     # --- Final DataFrame Preparation and Duplicate Analysis ---
#     if master_df.empty and not os.path.exists(OUTPUT_CSV_FILE):
#         print("No data to analyze or save. Exiting.")
#         return
        
#     # Ensure master_df is the most up-to-date from file, especially if no new images were processed
#     if os.path.exists(OUTPUT_CSV_FILE):
#         try:
#             print("Loading final dataset for duplicate analysis...")
#             master_df = pd.read_csv(OUTPUT_CSV_FILE)
#              # Ensure essential columns exist
#             if 'Image Name' not in master_df.columns or 'Hash Algorithm' not in master_df.columns or 'Hash Value' not in master_df.columns:
#                 print("CSV file is missing required columns. Cannot perform duplicate analysis.")
#                 return
#         except Exception as e:
#             print(f"Could not reload CSV for final analysis: {e}. Attempting with in-memory data if available.")
#             if master_df.empty: # if in-memory also empty
#                 return

#     if master_df.empty:
#         print("No hash data available for duplicate analysis. Exiting.")
#         return

#     print(f"Total unique images in dataset: {master_df['Image Name'].nunique()}")
#     print("Identifying duplicates based on all processed images...")
#     analysis_start_time = time.time()

#     hashes_by_algo_and_value = {
#         algo_name: master_df[master_df['Hash Algorithm'] == algo_name]
#                    .groupby('Hash Value')['Image Name']
#                    .apply(list)
#                    .to_dict()
#         for algo_name in HASH_ALGORITHMS.keys()
#     }

#     def find_row_duplicates(row):
#         algo = row['Hash Algorithm']
#         val = row['Hash Value']
#         current_name = row['Image Name']
#         if algo in hashes_by_algo_and_value and val in hashes_by_algo_and_value[algo]:
#             all_with_hash = hashes_by_algo_and_value[algo][val]
#             duplicates = [name for name in all_with_hash if name != current_name]
#             return duplicates if duplicates else None
#         return None

#     master_df['Duplicates'] = master_df.apply(find_row_duplicates, axis=1)
    
#     print(f"Duplicate analysis completed in {time.time() - analysis_start_time:.2f} seconds.")

#     print("\nSaving final DataFrame with duplicate information...")
#     robust_df_save(master_df, OUTPUT_CSV_FILE)

#     # --- Optional: Print summary of duplicates found ---
#     print("\n--- Duplicate Summary ---")
#     for algo_name in HASH_ALGORITHMS.keys():
#         # print(f"\nAlgorithm: {algo_name}")
#         algo_df = master_df[master_df['Hash Algorithm'] == algo_name]
#         duplicate_sets_for_algo = algo_df[algo_df['Duplicates'].apply(lambda x: x is not None and len(x) > 0)]
        
#         if not duplicate_sets_for_algo.empty:
#             # Create unique sets of duplicate groups
#             unique_groups = set()
#             for _, row in duplicate_sets_for_algo.iterrows():
#                 # A group includes the image itself and its duplicates, sorted for canonical representation
#                 group = frozenset(sorted([row['Image Name']] + row['Duplicates']))
#                 unique_groups.add(group)
            
#         #     if unique_groups:
#         #         # for group in sorted(list(unique_groups), key=lambda s: (len(s), sorted(list(s))[0])): # Sort groups for consistent output
#         #             # print(f"  - Set of duplicates: {sorted(list(group))}")
#         #     else:
#         #          print("  No unique duplicate sets found for this algorithm (after processing).") # Should not happen if duplicate_sets_for_algo not empty
#         # else:
#         #     print("  No duplicates found for this algorithm.")

#     total_time = time.time() - script_start_time
#     print(f"\nTotal script execution time: {total_time:.2f} seconds.")

# if __name__ == '__main__':
#     main()

### Inspect duplicates csv

In [2]:
# import numpy as np
# import pandas as pd
# import ast # For safely evaluating string representations of lists
# import io

# df = pd.read_csv("image_hashes_and_duplicates.csv")

# # view only non Nan duplicates
# df = df[df['Duplicates'].notna()]
# df = df[df["Hash Algorithm"].isin(["sha256", "dhash"])]

# print(df['Hash Algorithm'].value_counts())

# # GET list of duplicate images

# def get_final_duplicate_groups(df: pd.DataFrame) -> list[list[str]]:
#     """
#     Processes a DataFrame containing image duplication information to find
#     unique, consolidated groups of duplicate images.

#     Args:
#         df: Pandas DataFrame with columns including 'Image Name' and 'Duplicates'.
#             'Duplicates' column should contain a string representation of a list
#             of duplicate image filenames, or be NaN/empty if no duplicates
#             were found by that specific hash.

#     Returns:
#         A list of lists, where each inner list contains unique image filenames
#         that form a duplicate group. Inner lists are sorted, and the outer
#         list is sorted by the first element of each inner list for consistency.
#     """
#     initial_groups = []

#     for index, row in df.iterrows():
#         current_image = row['Image Name']
#         duplicates_val = row['Duplicates']
#         parsed_duplicates_list = []

#         if pd.notna(duplicates_val):
#             if isinstance(duplicates_val, str):
#                 # Clean up the string: remove potential outer quotes if any, then strip whitespace
#                 temp_str = duplicates_val.strip()
#                 if temp_str.startswith('"') and temp_str.endswith('"'): # Handle if CSV read quotes as part of string
#                     temp_str = temp_str[1:-1]
#                 temp_str = temp_str.strip()
                
#                 if temp_str and temp_str != "[]": # Ensure it's not empty or just "[]"
#                     try:
#                         evaluated_val = ast.literal_eval(temp_str)
#                         if isinstance(evaluated_val, list):
#                             # Filter to ensure all items in the list are strings (filenames)
#                             parsed_duplicates_list = [item for item in evaluated_val if isinstance(item, str)]
#                     except (ValueError, SyntaxError, TypeError):
#                         print(f"Warning: Could not parse 'Duplicates' string: '{duplicates_val}' for image '{current_image}' at index {index}. Skipping this entry for group formation.")
#             elif isinstance(duplicates_val, list): # If it's already a list (e.g., if DataFrame was constructed differently)
#                  parsed_duplicates_list = [item for item in duplicates_val if isinstance(item, str)]


#         # Create a group including the current image and its parsed duplicates
#         # Only proceed if there are actual duplicates found for this entry, or if the group would be non-trivial
#         current_group_set = set()
#         if isinstance(current_image, str): # Ensure current_image is a string
#             current_group_set.add(current_image)
        
#         if parsed_duplicates_list:
#             current_group_set.update(parsed_duplicates_list)

#         # Only add the group if it contains more than one image (i.e., actual duplicates)
#         # or if it's a single image that might merge with other groups later.
#         # For this problem, we are interested in sets formed by row['Image Name'] + row['Duplicates']
#         if parsed_duplicates_list: # Meaning this row explicitly states duplicates
#             initial_groups.append(current_group_set)


#     if not initial_groups:
#         return []

#     # Merge overlapping sets
#     # Algorithm: Iterate through initial_groups. For each group, try to merge it with existing
#     #            groups in final_merged_sets. If it overlaps with multiple, merge all involved.
#     final_merged_sets = []
#     for group_to_add in initial_groups:
#         if not group_to_add: # Skip empty sets if any were formed
#             continue
            
#         overlapping_indices = []
#         for i, existing_set in enumerate(final_merged_sets):
#             if not group_to_add.isdisjoint(existing_set): # Check for intersection
#                 overlapping_indices.append(i)
        
#         if not overlapping_indices:
#             # No overlap, add as a new set
#             final_merged_sets.append(group_to_add.copy())
#         else:
#             # Overlaps with one or more existing sets
#             merged_set = group_to_add.copy()
#             # Iterate in reverse to safely pop elements
#             for i in sorted(overlapping_indices, reverse=True):
#                 merged_set.update(final_merged_sets.pop(i))
#             final_merged_sets.append(merged_set)
            
#     # Convert sets to sorted lists
#     result_list_of_lists = [sorted(list(s)) for s in final_merged_sets if s] # Ensure no empty sets slip through

#     # Sort the outer list for deterministic output (e.g., by the first element of each inner list)
#     result_list_of_lists.sort(key=lambda x: x[0] if x else "")

#     return result_list_of_lists

# # Load the example data into a DataFrame
# df_example = df

# # Get the consolidated duplicate groups
# final_duplicate_groups = list(get_final_duplicate_groups(df_example.copy())) # Use .copy() if df might be modified

## Move unecessary duplicates

In [3]:
# import json
# import os
# import shutil

# # --- Configuration ---
# # TODO: Update these paths before running
# SOURCE_IMAGE_DIR = "./output" # e.g., "data/all_images"
# DUPLICATES_DEST_DIR = "./duplicates" # e.g., "data/moved_duplicates"

# # Example: final_duplicate_groups (replace with your actual list)
# # final_duplicate_groups = [
# #  ['0013dfe43ba6a59c437cc173685a3b6e.png', '7f58a1ef040408dfdf71ff82ec6cb43d.png'],
# #  ['001b9ee7bf40b962c1019fd029f6ce87.png', '1ff9e419afa6db9ab683b5e1846bc803.png', 'bc03f1381312769a5dd493d989779562.png'],
# #  # ... more groups
# # ]
# # Make sure 'final_duplicate_groups' is defined in your notebook before this script.


# def get_json_path(image_filename: str, directory: str) -> str:
#     """Constructs the path to the JSON file corresponding to an image."""
#     base_name, _ = os.path.splitext(image_filename)
#     return os.path.join(directory, base_name + ".json")

# def load_json_data(json_filepath: str) -> dict | None:
#     """Loads JSON data from a file."""
#     if not os.path.exists(json_filepath):
#         print(f"Warning: JSON file not found: {json_filepath}")
#         return None
#     try:
#         with open(json_filepath, 'r') as f:
#             return json.load(f)
#     except json.JSONDecodeError:
#         print(f"Warning: Could not decode JSON from file: {json_filepath}")
#         return None
#     except Exception as e:
#         print(f"Warning: Error loading JSON file {json_filepath}: {e}")
#         return None

# def is_label_descriptive(label: str | None) -> bool:
#     """Checks if an image class label is descriptive."""
#     if label is None:
#         return False # No label is not descriptive
#     label_lower = label.lower()
#     if label_lower == "0_unknown" or label_lower.endswith("_unknown"):
#         return False
#     return True

# def get_json_informativeness(json_data: dict | None) -> tuple[int, bool, str | None]:
#     """
#     Evaluates the informativeness of a JSON file.
#     Returns a tuple: (number_of_shapes, is_descriptive_label, image_class_label).
#     Higher number of shapes is better. Descriptive label is better.
#     """
#     if json_data is None:
#         return (0, False, None) # Least informative

#     num_shapes = len(json_data.get("shapes", []))
#     image_class_label = json_data.get("image_class_label")
#     descriptive_label = is_label_descriptive(image_class_label)
    
#     return (num_shapes, descriptive_label, image_class_label)

# def move_file_pair(image_filename: str, source_dir: str, dest_dir: str):
#     """Moves an image and its corresponding JSON file."""
#     base_name, img_ext = os.path.splitext(image_filename)
#     json_filename = base_name + ".json"

#     source_img_path = os.path.join(source_dir, image_filename)
#     source_json_path = os.path.join(source_dir, json_filename)
#     dest_img_path = os.path.join(dest_dir, image_filename)
#     dest_json_path = os.path.join(dest_dir, json_filename)

#     files_moved_count = 0
#     if os.path.exists(source_img_path):
#         try:
#             shutil.move(source_img_path, dest_img_path)
#             # print(f"Moved image: {image_filename} to {dest_dir}")
#             files_moved_count +=1
#         except Exception as e:
#             print(f"Error moving image {source_img_path}: {e}")
#     else:
#         print(f"Warning: Source image not found, cannot move: {source_img_path}")
        
#     if os.path.exists(source_json_path):
#         try:
#             shutil.move(source_json_path, dest_json_path)
#             # print(f"Moved JSON: {json_filename} to {dest_dir}")
#             files_moved_count +=1
#         except Exception as e:
#             print(f"Error moving JSON {source_json_path}: {e}")
#     else:
#         # This might be acceptable if the image didn't have a JSON, or it was already handled
#         print(f"Warning: Source JSON not found, cannot move: {source_json_path}")
#     return files_moved_count


# def process_duplicate_groups(duplicate_groups: list[list[str]], source_dir: str, duplicates_dir: str):
#     """
#     Processes groups of duplicate images, keeps the most informative one,
#     and moves the others to the duplicates_dir.
#     """
#     if not os.path.exists(duplicates_dir):
#         os.makedirs(duplicates_dir)
#         print(f"Created duplicates directory: {duplicates_dir}")

#     if not os.path.exists(source_dir):
#         print(f"Error: Source directory '{source_dir}' does not exist. Please check the path.")
#         return
        
#     total_files_to_move = 0
#     total_files_moved = 0

#     for i, group in enumerate(duplicate_groups):
#         print(f"\nProcessing group {i+1}/{len(duplicate_groups)}: {group}")
#         if not group or len(group) < 2:
#             print(f"Skipping group {i+1} as it has less than two items or is empty.")
#             continue

#         best_image_filename = None
#         # Initialize with least informative: (num_shapes, is_descriptive, label_string)
#         best_informativeness = (-1, False, None) 

#         candidate_info = []

#         for image_filename in group:
#             json_filepath = get_json_path(image_filename, source_dir)
#             json_data = load_json_data(json_filepath)
            
#             if json_data is None and not os.path.exists(json_filepath):
#                 # If JSON doesn't exist, it's less informative than one that does.
#                 # Still, we need to track it to potentially keep the image if all others also lack JSONs
#                 # or if it's the only one left by some chance.
#                 # This case is treated as (0 shapes, not descriptive).
#                 current_informativeness = (0, False, None)
#                 print(f"  Image '{image_filename}': No JSON found or JSON is invalid. Treated as least informative.")
#             elif json_data is None and os.path.exists(json_filepath):
#                  current_informativeness = (0, False, None) # JSON exists but is invalid
#                  print(f"  Image '{image_filename}': JSON found but invalid. Treated as least informative.")
#             else:
#                 current_informativeness = get_json_informativeness(json_data)
#                 print(f"  Image '{image_filename}': Shapes={current_informativeness[0]}, DescriptiveLabel={current_informativeness[1]}, Label='{current_informativeness[2]}'")

#             candidate_info.append({
#                 "filename": image_filename,
#                 "informativeness": current_informativeness,
#                 "json_exists_and_valid": json_data is not None
#             })

#         # Determine the best candidate
#         # The first image in the original `group` list acts as a tie-breaker if informativeness is identical.
#         # We iterate through candidates in the order they appeared in the `group`.
        
#         # Initialize best_image_filename with the first item, in case all have no JSON or are equally bad.
#         best_image_filename = candidate_info[0]["filename"]
#         best_informativeness = candidate_info[0]["informativeness"]
#         best_json_valid = candidate_info[0]["json_exists_and_valid"]

#         for candidate in candidate_info[1:]: # Start from the second candidate
#             current_filename = candidate["filename"]
#             current_informativeness = candidate["informativeness"]
#             current_json_valid = candidate["json_exists_and_valid"]

#             # Priority:
#             # 1. More shapes is better.
#             # 2. Descriptive label is better than non-descriptive/missing.
#             # 3. If all above are equal, prefer one with a valid JSON over one without.
#             # 4. If all above are equal, the first one encountered (implicit from loop order) is kept.

#             if current_informativeness[0] > best_informativeness[0]: # More shapes
#                 best_image_filename = current_filename
#                 best_informativeness = current_informativeness
#                 best_json_valid = current_json_valid
#             elif current_informativeness[0] == best_informativeness[0]: # Equal shapes
#                 if current_informativeness[1] and not best_informativeness[1]: # Current is descriptive, best is not
#                     best_image_filename = current_filename
#                     best_informativeness = current_informativeness
#                     best_json_valid = current_json_valid
#                 elif current_informativeness[1] == best_informativeness[1]: # Both descriptive or both non-descriptive
#                     # If labels are equivalent in descriptiveness, prefer one with a valid JSON
#                     if current_json_valid and not best_json_valid:
#                         best_image_filename = current_filename
#                         best_informativeness = current_informativeness
#                         best_json_valid = current_json_valid
#                     # If both have valid JSONs or both don't, and labels/shapes are equal,
#                     # the existing 'best_image_filename' (which was earlier in the list) is kept.


#         print(f"  -> Keeping: '{best_image_filename}' (Shapes={best_informativeness[0]}, Descriptive={best_informativeness[1]}, Label='{best_informativeness[2]}')")

#         # Move other files in the group
#         for image_filename in group:
#             if image_filename != best_image_filename:
#                 print(f"  -> Moving: '{image_filename}'")
#                 total_files_to_move += 2 # For image and JSON
#                 moved_count = move_file_pair(image_filename, source_dir, duplicates_dir)
#                 total_files_moved += moved_count
#             else:
#                 # Ensure the JSON for the "best" image actually exists if we determined it should.
#                 # This is more of a sanity check, as 'load_json_data' handles non-existence.
#                 best_json_path = get_json_path(best_image_filename, source_dir)
#                 if best_json_valid and not os.path.exists(best_json_path):
#                      print(f"Warning: The selected best image '{best_image_filename}' was expected to have a valid JSON, but '{best_json_path}' was not found during the move phase. This shouldn't happen if logic is correct.")
#                 elif not best_json_valid and os.path.exists(best_json_path):
#                      # This could happen if the JSON was present but unreadable. It will remain.
#                      pass


#     print(f"\n--- Processing Complete ---")
#     print(f"Attempted to move up to {total_files_to_move} files (images and JSONs).")
#     print(f"Actually moved {total_files_moved} files.")
#     remaining_files_in_source = len([name for name in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, name))])
#     files_in_duplicates_dir = len([name for name in os.listdir(duplicates_dir) if os.path.isfile(os.path.join(duplicates_dir, name))])
#     print(f"Files remaining in source directory ('{source_dir}'): {remaining_files_in_source}")
#     print(f"Files in duplicates directory ('{duplicates_dir}'): {files_in_duplicates_dir}")


# # --- Main execution ---
# if __name__ == "__main__":
#     # This is where you would load your `final_duplicate_groups`
#     # For testing, you can create a dummy CSV and DataFrame like in your example,
#     # or directly define `final_duplicate_groups`.

#     # Ensure `final_duplicate_groups` is defined.
#     # Example (if you're not running this directly in the notebook after your code):
#     # final_duplicate_groups = [
#     #     ['image1_dupA.png', 'image1_dupB.png', 'image1_original.png'],
#     #     ['image2_better.png', 'image2_worse.png']
#     # ]
#     # Make sure SOURCE_IMAGE_DIR contains dummy files like:
#     # image1_dupA.png, image1_dupA.json
#     # image1_dupB.png, image1_dupB.json
#     # image1_original.png, image1_original.json
#     # etc. with varying JSON content.

#     # --- RUN THE SCRIPT ---
#     # Ensure `final_duplicate_groups` is defined from your previous notebook cell
#     if 'final_duplicate_groups' in locals() or 'final_duplicate_groups' in globals():
#         if not SOURCE_IMAGE_DIR.startswith("path/to/your") and not DUPLICATES_DEST_DIR.startswith("path/to/your"):
#             process_duplicate_groups(final_duplicate_groups, SOURCE_IMAGE_DIR, DUPLICATES_DEST_DIR)
#         else:
#             print("ERROR: Please update SOURCE_IMAGE_DIR and DUPLICATES_DEST_DIR variables in the script.")
#             print("SOURCE_IMAGE_DIR is currently:", SOURCE_IMAGE_DIR)
#             print("DUPLICATES_DEST_DIR is currently:", DUPLICATES_DEST_DIR)

#     else:
#         print("Error: `final_duplicate_groups` is not defined. Please ensure it's generated before running this script.")

## show duplicate images

In [4]:
# import os
# import matplotlib.pyplot as plt
# from PIL import Image, UnidentifiedImageError # For loading images and handling errors
# import numpy as np # For more robust Axes handling

# def visualize_duplicate_groups(
#     duplicate_groups: list[list[str]],
#     image_folder_path: str,
#     max_images_per_row: int = 4,
#     max_groups_to_display: int = None,
#     figsize_per_image: tuple[float, float] = (3, 3)
# ) -> None:
#     """
#     Visualizes groups of duplicate images by displaying them in a grid.

#     Args:
#         duplicate_groups: A list of lists, where each inner list contains
#                           filenames of images considered duplicates.
#         image_folder_path: The base path to the folder containing the image files.
#         max_images_per_row: Maximum number of images to display in a single row
#                             within the grid for each group.
#         max_groups_to_display: Optional. The maximum number of duplicate groups
#                                to display. If None, all groups are processed.
#                                This is useful to prevent opening too many plot windows.
#         figsize_per_image: Tuple (width, height) in inches for each individual
#                            image subplot. The total figure size will be scaled based
#                            on this and the number of images.
#     """
#     if not duplicate_groups:
#         print("No duplicate groups to visualize.")
#         return

#     num_groups_actually_displayed = 0
#     for i, group in enumerate(duplicate_groups):
#         if max_groups_to_display is not None and num_groups_actually_displayed >= max_groups_to_display:
#             print(f"\nReached maximum number of groups to display ({max_groups_to_display}).")
#             break

#         if not group or len(group) < 2:
#             # Optionally print a message if you want to know about skipped small/empty groups
#             # print(f"Skipping Group {i+1}: Contains fewer than 2 images or is empty.")
#             continue

#         print(f"\nProcessing Duplicate Group {i+1} (contains {len(group)} candidate images):")

#         images_to_display = []
#         image_titles = []
#         loaded_image_objects = [] # To keep track of PIL Image objects for closing

#         for filename in group:
#             img_path = os.path.join(image_folder_path, filename)
#             try:
#                 if not os.path.exists(img_path):
#                     print(f"  WARNING: Image file not found at '{img_path}'. Skipping.")
#                     continue
#                 img = Image.open(img_path)
#                 images_to_display.append(img.copy()) # Work with a copy
#                 image_titles.append(filename)
#                 loaded_image_objects.append(img) # Add original to close later
#             except UnidentifiedImageError:
#                 print(f"  WARNING: Could not open or identify image '{img_path}'. It might be corrupted or not a valid image format. Skipping.")
#             except FileNotFoundError: # Should be caught by os.path.exists, but as a safeguard
#                 print(f"  WARNING: Image file not found (FileNotFoundError) at '{img_path}'. Skipping.")
#             except Exception as e:
#                 print(f"  WARNING: An unexpected error occurred loading image '{img_path}': {e}. Skipping.")

#         if len(images_to_display) < 1:
#             print(f"  No valid images could be loaded for Group {i+1}. Skipping visualization for this group.")
#             # Close any images that were successfully loaded before this group was skipped
#             for img_obj in loaded_image_objects:
#                 img_obj.close()
#             continue

#         num_images = len(images_to_display)
#         n_cols = min(num_images, max_images_per_row)
#         n_cols = max(1, n_cols) # Ensure at least 1 column
#         n_rows = (num_images + n_cols - 1) // n_cols  # Ceiling division

#         fig_width = n_cols * figsize_per_image[0]
#         fig_height = n_rows * figsize_per_image[1]

#         fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), squeeze=False)
#         # 'squeeze=False' ensures 'axes' is always a 2D array, simplifying indexing.

#         fig.suptitle(f"Duplicate Group {i+1} ({num_images} of {len(group)} images shown)", fontsize=16)

#         ax_flat = axes.flatten() # Flatten the 2D array of axes for easy iteration

#         for idx, (img_data, title) in enumerate(zip(images_to_display, image_titles)):
#             if idx < len(ax_flat):
#                 ax_flat[idx].imshow(img_data)
#                 ax_flat[idx].set_title(title, fontsize=10)
#                 ax_flat[idx].axis('off')  # Hide axis ticks and labels

#         # Turn off unused subplots if the grid is not perfectly filled
#         for idx in range(num_images, len(ax_flat)):
#             ax_flat[idx].axis('off')

#         plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make space for suptitle
#         plt.show()  # In a script, this blocks until the window is closed.
#                     # In Jupyter Notebook, displays inline.

#         num_groups_actually_displayed += 1

#         # Close all PIL Image objects for the current group after displaying
#         for img_obj in loaded_image_objects:
#             img_obj.close()
#         for img_data in images_to_display: # If img.copy() was used and stored here
#             if hasattr(img_data, 'close'):
#                 img_data.close()


#     if num_groups_actually_displayed == 0 and any(g for g in duplicate_groups if len(g or []) >=2):
#         print("\nNo groups were displayed. This might be due to all groups having fewer than 2 images, issues loading all images, or all groups being skipped.")
#     elif num_groups_actually_displayed > 0 :
#         print(f"\nDisplayed {num_groups_actually_displayed} group(s).")

# # --- How to use this function with your data ---
# #
# # 1. Make sure the function `visualize_duplicate_groups` is defined (as above).
# #
# # 2. You need to have your `final_duplicate_groups` variable populated.
# #    This variable should be a list of lists, where each inner list
# #    contains the filenames of images identified as duplicates.
# #    For example:
# #    final_duplicate_groups = [
# #        ['imageA_01.png', 'imageA_02.png', 'imageA_03.png'], # Group 1
# #        ['imageB_01.png', 'imageB_02.png'],                 # Group 2
# #        # ... more groups
# #    ]
# #
# # 3. You need to specify the path to the folder where these images are stored.
# #    For example:
# #    my_actual_image_folder_path = "/path/to/your/image_directory"
# #    # Or on Windows:
# #    # my_actual_image_folder_path = r"C:\path\to\your\image_directory"
# #
# # 4. Call the function:
# #    if 'final_duplicate_groups' in locals() and 'my_actual_image_folder_path' in locals():
# #        if final_duplicate_groups and my_actual_image_folder_path:
# #            print("Starting visualization of your duplicate image groups...")
# #            visualize_duplicate_groups(
# #                final_duplicate_groups,
# #                my_actual_image_folder_path,
# #                max_images_per_row=3,        # Optional: Adjust as needed
# #                max_groups_to_display=None,  # Optional: Set to a number e.g. 5, or None for all
# #                figsize_per_image=(3, 3)     # Optional: Adjust as needed
# #            )
# #        else:
# #            print("Please ensure 'final_duplicate_groups' is not empty and 'my_actual_image_folder_path' is set.")
# #    else:
# #        print("Please define 'final_duplicate_groups' and 'my_actual_image_folder_path' before calling the visualization.")

# # Ensure the 'visualize_duplicate_groups' function is defined above this point.
# # import os # Already imported within visualize_duplicate_groups if needed there,
#            # but good practice to have it if you use os.path.exists here too.
# # import matplotlib.pyplot as plt # Already imported within visualize_duplicate_groups
# # from PIL import Image, UnidentifiedImageError # Already imported
# # import numpy as np # Already imported

# # --- STEP 1: Assume your 'final_duplicate_groups' object is already populated ---
# # This object should be a list of lists of image filenames, for example:
# #
# # final_duplicate_groups = [
# #     ['image_A_copy1.jpg', 'image_A_original.jpg', 'image_A_variant.png'],
# #     ['photo_XYZ.png', 'photo_XYZ_duplicate.png'],
# #     # ... more groups ...
# # ]
# #
# # Make sure this variable exists and contains the data from your previous processing steps.


# # --- STEP 2: Define the path to the folder containing your actual images ---
# #
# # !! IMPORTANT !!
# # You MUST replace the placeholder path below with the ACTUAL path
# # to the directory where your image files are stored on your computer.
# #
# # For example, if your images are in "/home/user/pictures/my_project_images":
# # my_image_directory = "/home/user/pictures/my_project_images"
# #
# # Or, if on Windows, they are in "D:\photos\dataset":
# # my_image_directory = r"D:\photos\dataset"

# my_image_directory = "./output"


# # --- STEP 3: Call the visualization function ---

# # Basic check to ensure the variables are ready
# if 'final_duplicate_groups' in locals() and isinstance(final_duplicate_groups, list):
#     if my_image_directory == "/replace/with/actual/path/to/your/image_folder":
#         print("⚠️ WARNING: Please update 'my_image_directory' with the actual path to your image folder.")
#     elif not os.path.exists(my_image_directory):
#         print(f"⛔ ERROR: The specified image folder path does not exist: {my_image_directory}")
#         print("Please ensure 'my_image_directory' points to the correct location of your images.")
#     elif not final_duplicate_groups:
#         print("Information: 'final_duplicate_groups' is empty. No duplicate groups to visualize.")
#     else:
#         print(f"Starting visualization for image groups found in: {my_image_directory}")
#         visualize_duplicate_groups(
#             duplicate_groups=final_duplicate_groups,
#             image_folder_path=my_image_directory,
#             max_images_per_row=3,        # Optional: Adjust how many images appear side-by-side
#             max_groups_to_display=20,     # Optional: Limit to e.g., first 10 groups, or None for all
#             figsize_per_image=(3, 3)     # Optional: Adjust the display size of each image
#         )
# else:
#     print("⛔ ERROR: 'final_duplicate_groups' is not defined or is not a list.")
#     print("       Please ensure your previous code successfully created this object.")
#     print("       If 'final_duplicate_groups' is defined, ensure it's a list of lists as expected.")
# #

### Move json files with no images

In [8]:
import os
import shutil

# --- Configuration ---
# TODO: Update these paths before running
SOURCE_DIR = "./output"  # Directory to scan
ORPHANED_JSON_DEST_DIR = "./duplicates" # Directory to move orphaned JSONs

# Common image extensions to check for (ensure lowercase)
IMAGE_EXTENSIONS = ['.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp']

def move_orphaned_json_files_optimized(source_directory: str, orphaned_dest_directory: str, img_extensions: list[str]):
    """
    Moves JSON files from source_directory to orphaned_dest_directory
    if no corresponding image file (with a common extension) exists.
    This version is optimized to reduce file system checks.
    """
    if not os.path.exists(source_directory):
        print(f"Error: Source directory '{source_directory}' does not exist.")
        return

    if not os.path.exists(orphaned_dest_directory):
        try:
            os.makedirs(orphaned_dest_directory)
            print(f"Created directory for orphaned JSONs: '{orphaned_dest_directory}'")
        except OSError as e:
            print(f"Error: Could not create destination directory '{orphaned_dest_directory}': {e}")
            return

    image_basenames = set()
    json_files_to_process = []

    print(f"\nScanning '{source_directory}' to build file lists (optimized)...")

    # First pass: Collect all image basenames (without extension, lowercase)
    # and identify all JSON files (original case for moving).
    all_files = []
    try:
        all_files = os.listdir(source_directory)
    except OSError as e:
        print(f"Error listing files in '{source_directory}': {e}")
        return

    for filename in all_files:
        # Normalize to lowercase for extension and basename comparison
        name_lower = filename.lower()
        base_name_lower, ext_lower = os.path.splitext(name_lower)

        if ext_lower in img_extensions:
            # Check if the file actually exists (os.listdir might return names of deleted files in some edge cases)
            # and is a file (not a directory with an image-like extension)
            if os.path.isfile(os.path.join(source_directory, filename)):
                 image_basenames.add(base_name_lower)
        elif ext_lower == ".json":
            if os.path.isfile(os.path.join(source_directory, filename)):
                json_files_to_process.append(filename) # Store original filename

    print(f"Found {len(image_basenames)} unique image base names.")
    print(f"Found {len(json_files_to_process)} JSON files to check.")

    moved_count = 0
    orphaned_json_found_count = 0

    # Second pass: Check JSON files against the image basenames set
    for json_filename in json_files_to_process:
        json_base_name_lower, _ = os.path.splitext(json_filename.lower())

        if json_base_name_lower not in image_basenames:
            orphaned_json_found_count += 1
            source_json_path = os.path.join(source_directory, json_filename)
            dest_json_path = os.path.join(orphaned_dest_directory, json_filename)
            try:
                shutil.move(source_json_path, dest_json_path)
                # print(f"Moved orphaned JSON: '{json_filename}' to '{orphaned_dest_directory}'")
                moved_count += 1
            except FileNotFoundError:
                 print(f"Warning: JSON file '{json_filename}' was listed but not found during move operation. It might have been moved or deleted by another process.")
            except Exception as e:
                print(f"Error moving JSON file '{json_filename}': {e}")
        
    print(f"\n--- Orphaned JSON Scan Complete (Optimized) ---")
    print(f"Total JSON files checked: {len(json_files_to_process)}.")
    print(f"Identified {orphaned_json_found_count} as potentially orphaned.")
    print(f"Successfully moved {moved_count} orphaned JSON files to '{orphaned_dest_directory}'.")

# --- Main execution ---
if __name__ == "__main__": # Good practice for notebooks too
    # Ensure SOURCE_DIR and ORPHANED_JSON_DEST_DIR are correctly set
    if not SOURCE_DIR.startswith("path/to/your") and \
       not ORPHANED_JSON_DEST_DIR.startswith("path/to/your"):
        move_orphaned_json_files_optimized(SOURCE_DIR, ORPHANED_JSON_DEST_DIR, IMAGE_EXTENSIONS)
    else:
        print("ERROR: Please update SOURCE_DIR and ORPHANED_JSON_DEST_DIR variables in the script.")
        print("SOURCE_DIR is currently:", SOURCE_DIR)
        print("ORPHANED_JSON_DEST_DIR is currently:", ORPHANED_JSON_DEST_DIR)


Scanning './output' to build file lists (optimized)...
Found 54421 unique image base names.
Found 54421 JSON files to check.

--- Orphaned JSON Scan Complete (Optimized) ---
Total JSON files checked: 54421.
Identified 0 as potentially orphaned.
Successfully moved 0 orphaned JSON files to './duplicates'.


### Copy files fast with the following cell

In [6]:
# import os
# import shutil
# import multiprocessing
# import time # argparse is removed

# def copy_file_worker(src_path, dst_path):
#     """
#     Worker function to copy a single file.
#     This function is intended to be executed by a process in a multiprocessing pool.

#     Args:
#         src_path (str): Path to the source file.
#         dst_path (str): Path to the destination file where the source file will be copied.

#     Returns:
#         tuple: A tuple containing:
#             - src_path (str): The path of the source file processed.
#             - success_status (bool): True if copying was successful, False otherwise.
#             - error_message (str or None): An error message string if copying failed, None otherwise.
#     """
#     try:
#         # Ensure the destination directory for the file exists.
#         os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        
#         # shutil.copy2 attempts to preserve all metadata (like timestamps, permissions)
#         shutil.copy2(src_path, dst_path)
        
#         # print(f"Copied: {src_path} -> {dst_path}") # Uncomment for verbose output
#         return src_path, True, None
#     except Exception as e:
#         # print(f"Error copying {src_path} to {dst_path}: {e}") # Uncomment for immediate error feedback
#         return src_path, False, str(e)

# def parallel_copy_folder(source_folder, destination_folder, num_processes=None):
#     """
#     Copies a folder from a source path to a destination path in parallel.
#     It recreates the directory structure from the source in the destination
#     and then copies files using multiple processes.

#     Args:
#         source_folder (str): The path to the source folder to be copied.
#         destination_folder (str): The path to the destination folder. If it doesn't exist,
#                                   it will be created. If it exists, files will be copied into it.
#         num_processes (int, optional): The number of worker processes to use for copying files.
#                                        Defaults to the number of CPU cores available (os.cpu_count()).
#     """
#     start_time = time.time()

#     # --- 1. Validate Source Folder ---
#     if not os.path.exists(source_folder):
#         print(f"Error: Source folder '{source_folder}' does not exist.")
#         return
#     if not os.path.isdir(source_folder):
#         print(f"Error: Source '{source_folder}' is not a directory.")
#         return

#     # --- 2. Prepare Destination Folder ---
#     source_folder_abs = os.path.abspath(source_folder)
#     destination_folder_abs = os.path.abspath(destination_folder)

#     if os.path.exists(destination_folder_abs):
#         if not os.path.isdir(destination_folder_abs):
#             print(f"Error: Destination '{destination_folder_abs}' exists and is not a directory.")
#             return
#         # print(f"Warning: Destination folder '{destination_folder_abs}' already exists. Files may be overwritten.")
#     else:
#         try:
#             os.makedirs(destination_folder_abs, exist_ok=True)
#             print(f"Created destination folder: {destination_folder_abs}")
#         except OSError as e:
#             print(f"Error creating destination folder '{destination_folder_abs}': {e}")
#             return

#     # --- 3. Collect File Copy Tasks and Create Directory Structure ---
#     tasks = [] 

#     print(f"Scanning source folder: {source_folder_abs}...")
#     for dirpath, dirnames, filenames in os.walk(source_folder_abs):
#         relative_path = os.path.relpath(dirpath, source_folder_abs)
        
#         if relative_path == ".":
#             dest_dirpath = destination_folder_abs
#         else:
#             dest_dirpath = os.path.join(destination_folder_abs, relative_path)

#         if not os.path.exists(dest_dirpath):
#             try:
#                 os.makedirs(dest_dirpath, exist_ok=True)
#             except OSError as e:
#                 print(f"Error creating directory {dest_dirpath}: {e}")
#                 continue 

#         for filename in filenames:
#             src_file_path = os.path.join(dirpath, filename)
#             dst_file_path = os.path.join(dest_dirpath, filename)
#             tasks.append((src_file_path, dst_file_path))

#     if not tasks:
#         print("No files found to copy in the source folder.")
#         duration = time.time() - start_time
#         print(f"Operation completed in {duration:.2f} seconds (no files to copy).")
#         return

#     print(f"Found {len(tasks)} files to copy. Starting parallel copy operation...")

#     # --- 4. Execute Copy Tasks in Parallel ---
#     if num_processes is None:
#         num_processes = os.cpu_count() 
    
#     actual_num_processes = min(num_processes, len(tasks)) 
    
#     print(f"Using {actual_num_processes} processes for copying.")

#     results = []
#     # Important: On some systems, especially Windows, multiprocessing within a Jupyter Notebook
#     # might require the worker function and the pool execution to be guarded by `if __name__ == '__main__':`.
#     # However, for a single cell execution, this structure is generally fine.
#     # If you encounter issues with multiprocessing (e.g., hanging),
#     # you might need to move the worker function definition and the pool logic
#     # into a separate .py file and import it, or investigate platform-specific
#     # multiprocessing behavior in notebooks.
#     with multiprocessing.Pool(processes=actual_num_processes) as pool:
#         results = pool.starmap(copy_file_worker, tasks)

#     # --- 5. Report Results ---
#     successful_copies = 0
#     failed_copies_details = [] 

#     for src_path, success, error_msg in results:
#         if success:
#             successful_copies += 1
#         else:
#             failed_copies_details.append((src_path, error_msg))

#     end_time = time.time()
#     duration = end_time - start_time

#     print(f"\n--- Copy Operation Summary ---")
#     print(f"Total files scheduled for copy: {len(tasks)}")
#     print(f"Successfully copied files: {successful_copies}")
#     print(f"Failed to copy files: {len(failed_copies_details)}")
    
#     if failed_copies_details:
#         print("\nDetails of failed copies:")
#         for src, err in failed_copies_details:
#             print(f"  - File: {src}\n    Error: {err}")
            
#     print(f"\nTotal time taken for copy operation: {duration:.2f} seconds.")

# # --- Configuration for Jupyter Notebook ---
# # IMPORTANT: Define your source, destination, and number of processes here.
# # Make sure to use raw strings (r"...") or escape backslashes (\) for Windows paths.

# # Example for Linux/macOS:
# # MY_SOURCE_FOLDER = "/path/to/your/source_folder"
# # MY_DESTINATION_FOLDER = "/path/to/your/destination_folder"

# # Example for Windows:
# # MY_SOURCE_FOLDER = r"C:\path\to\your\source_folder"
# # MY_DESTINATION_FOLDER = r"C:\path\to\your\destination_folder"

# # Fill these in with your actual paths:
# MY_SOURCE_FOLDER = r"/blue/hulcr/gmarais/PhD/phase_1_data/20251505/output"  # e.g., r"/mnt/data/source_dir" or r"D:\source_data"
# MY_DESTINATION_FOLDER = r"/blue/hulcr/gmarais/data_backups/with_duplicates_20250515" # e.g., r"/mnt/data/destination_dir" or r"E:\backup_data"
# NUMBER_OF_PROCESSES = None  # Set to an integer (e.g., 4) or None to use all available CPU cores

# # --- Run the copy operation ---
# if MY_SOURCE_FOLDER == "REPLACE_WITH_YOUR_SOURCE_PATH" or \
#    MY_DESTINATION_FOLDER == "REPLACE_WITH_YOUR_DESTINATION_PATH":
#     print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#     print("!!! PLEASE UPDATE 'MY_SOURCE_FOLDER' AND 'MY_DESTINATION_FOLDER'          !!!")
#     print("!!! in the script before running to specify your actual paths.            !!!")
#     print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
# else:
#     print(f"Initiating folder copy from '{MY_SOURCE_FOLDER}' to '{MY_DESTINATION_FOLDER}'...")
#     if NUMBER_OF_PROCESSES:
#         print(f"User specified number of processes: {NUMBER_OF_PROCESSES}")
#     else:
#         print(f"Number of processes will default to available CPU cores.")

#     # Call the main function to perform the copy
#     # This part is no longer inside an `if __name__ == "__main__":` block
#     # as it's intended to be run directly in a notebook cell.
#     parallel_copy_folder(MY_SOURCE_FOLDER, MY_DESTINATION_FOLDER, NUMBER_OF_PROCESSES)

